In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
import pylab as pl

import numpy as np
import time
import os
#import re
import sys
import math
import pickle
import pandas as pd
import csv
from __future__ import division
import itertools

from skimage import io, filters, util, data, img_as_float
import scipy
import brewer2mpl

from sklearn.cluster import KMeans
from sklearn.decomposition import MiniBatchDictionaryLearning, SparseCoder, sparse_encode, PCA
from sklearn.feature_extraction.image import extract_patches_2d, PatchExtractor, reconstruct_from_patches_2d
#from sklearn.manifold import TSNE
from skimage.transform import downscale_local_mean

import skynet.pipeline
import skynet.utils
import skynet.dl_utils as dl
import skynet.viz as viz
import skynet.sparse_encoding as se

from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram, leaves_list

data_path = "/Users/don/Documents/patch_data/"
dl_path = "/Users/don/Documents/DL/"

#Tell numpy to skip division by zero in broadcasting
np.seterr(divide = 'ignore', invalid = 'ignore')

{'divide': 'ignore', 'invalid': 'ignore', 'over': 'ignore', 'under': 'ignore'}

In [2]:
# Load the desired dictionary(s)
#V_name_ls = ['V46a1e-1']
V_name_ls = ['V46a1e-1', 'V46a1e-2', 'V46a01', 'V46a02', 'V46a04', 'V46a08', 'V46a16', 'V46a32']
V_dict = {}

for name in V_name_ls:
    V_name = name+'.npy'
    V_data = np.load(dl_path+V_name)
    V_dict[name] = V_data[1]; V_params = V_data[0]
    print("Loaded %s: learning params = (%s, %s, %s(%s), %s)" %
         (name, V_params[0], V_params[1], V_params[2], V_params[3], V_params[4]))

Loaded V46a1e-1: learning params = (p1_tr.p, 300, lasso_lars(0.1), 64000)
Loaded V46a1e-2: learning params = (p1_tr.p, 300, lasso_lars(0.01), 64000)
Loaded V46a01: learning params = (p1_tr.p, 300, lasso_lars(1), 64000)
Loaded V46a02: learning params = (p1_tr.p, 300, lasso_lars(2), 64000)
Loaded V46a04: learning params = (p1_tr.p, 300, lasso_lars(4), 64000)
Loaded V46a08: learning params = (p1_tr.p, 300, lasso_lars(8), 64000)
Loaded V46a16: learning params = (p1_tr.p, 300, lasso_lars(16), 64000)
Loaded V46a32: learning params = (p1_tr.p, 300, lasso_lars(32), 64000)


In [3]:
# Load the appropriate data sets
d0 = pickle.load(open(data_path+'p1_tr.p',"rb"))
d1 = pickle.load(open(data_path+'p1_te.p',"rb"))

# Define sparse-encoding params
alfa_ls = [1,2,4,8,16,32] # sparse encoding params

In [4]:
# this block will sparse encode, evalute recon, and save the sparse encodings
# overriding the results each time
for V_nm in V_name_ls:
    print("Processing %s" % V_nm)
    # Sparse encoding
    print("Sparse encoding...")
    t0 = time.time()
    X_dict1 = se.patch_and_sparse_encode(d0, V_dict[V_nm], alfa_ls)
    X_dict2 = se.patch_and_sparse_encode(d1, V_dict[V_nm], alfa_ls)
    print("Sparse encoding done in %.2fs" % (time.time() - t0))

    # Reconstruction
    print("Reconstructing...")
    XV_dict1 = {}
    XV_dict2 = {}
    for alfa in alfa_ls:
        recon = np.dot(X_dict1[alfa], V_dict[V_nm])
        recon = (recon - recon.min())/(recon.max() - recon.min())
        XV_dict1[alfa] = recon

        recon = np.dot(X_dict2[alfa], V_dict[V_nm])
        recon = (recon - recon.min())/(recon.max() - recon.min())
        XV_dict2[alfa] = recon

    print("Recon error...")
    # Get reconstruction error
    err_arr_dict1 = {}
    err_arr_dict2 = {}
    err_tr_dict = {}
    err_te_dict = {}

    img_data_n = se.get_img_data(d0)
    for alfa in alfa_ls:
        err_arr, mu, sigma = viz.get_reconstruction_error(XV_dict1[alfa], img_data_n, alfa)
        err_arr_dict1[alfa] = err_arr
        loss_arr = viz.get_loss(X_dict1[alfa], XV_dict1[alfa], img_data_n, alfa)
        err_tr_dict[alfa]=[mu, sigma, np.average(loss_arr)]

    img_data_n = se.get_img_data(d1)
    for alfa in alfa_ls:
        err_arr, mu, sigma = viz.get_reconstruction_error(XV_dict2[alfa], img_data_n, alfa)
        err_arr_dict2[alfa] = err_arr
        loss_arr = viz.get_loss(X_dict2[alfa], XV_dict2[alfa], img_data_n, alfa)
        err_te_dict[alfa]=[mu, sigma, np.average(loss_arr)]

    err_dict = {'tr_err':err_tr_dict, 'te_err':err_te_dict}

    # Print minimal recon errors
    mean_tr_err = []
    mean_te_err = []
    for alfa in alfa_ls:
        mean_tr_err.append(err_tr_dict[alfa][0])
        mean_te_err.append(err_te_dict[alfa][0])
    mean_tr_err = []
    mean_te_err = []
    for alfa in alfa_ls:
        mean_tr_err.append([alfa, err_tr_dict[alfa][0]])
        mean_te_err.append([alfa, err_te_dict[alfa][0]])
    mean_tr_err = np.array(mean_tr_err)
    mean_te_err = np.array(mean_te_err)

    tr_min_idx = np.argmin(mean_tr_err[:,1])
    tr_min_alfa = mean_tr_err[tr_min_idx,0]
    tr_min = mean_tr_err[tr_min_idx,1]

    te_min_idx = np.argmin(mean_te_err[:,1])
    te_min_alfa = mean_te_err[te_min_idx,0]
    te_min = mean_te_err[te_min_idx,1]

    print("Smallest recon error: %.3f(alpha = %s)" % (tr_min, tr_min_alfa))
    print("Smallest recon error: %.3f(alpha = %s)" % (te_min, te_min_alfa))

    sps_dict1 = X_dict1.copy()
    for k in list(sps_dict1.keys()):
        sps_dict1[k] = list(sps_dict1[k])
    # Let's just hope that patch-level order is preserved
    sps_dict1['img_idx'] = list(d0['img_idx'])
    sps_dict1['moa'] = list(d0['moa'])
    sps_dict1['cpd'] = list(d0['cpd'])
    sps_dict1['cc'] = list(d0['cc'])
    sps_df1 = pd.DataFrame(sps_dict1)

    sps_dict2 = X_dict2.copy()
    for k in list(sps_dict2.keys()):
        sps_dict2[k] = list(sps_dict2[k])
    # Let's just hope that patch-level order is preserved
    sps_dict2['img_idx'] = list(d1['img_idx'])
    sps_dict2['moa'] = list(d1['moa'])
    sps_dict2['cpd'] = list(d1['cpd'])
    sps_dict2['cc'] = list(d1['cc'])
    sps_df2 = pd.DataFrame(sps_dict2)

    save_str = data_path + V_nm + '_lars.p'
    print(save_str)
    pickle.dump([sps_df1, sps_df2, err_dict], open(save_str, 'wb'))
    print("------------------------------")

Processing V46a1e-1
Sparse encoding...
Img data shape = (19200, 2700)
Dictionary shape = (300, 2700)
1, 218.51s
2, 114.36s
4, 69.97s
8, 31.00s
16, 12.84s
32, 5.40s
Img data shape = (4800, 2700)
Dictionary shape = (300, 2700)
1, 57.42s
2, 30.18s
4, 16.13s
8, 7.43s
16, 3.45s
32, 1.55s
Sparse encoding done in 569.69s
Reconstructing...
Recon error...
Smallest recon error: 2.632(alpha = 1.0)
Smallest recon error: 1.917(alpha = 1.0)
/Users/don/Documents/patch_data/V46a1e-1_lars.p
Processing V46a1e-2
Sparse encoding...
Img data shape = (19200, 2700)
Dictionary shape = (300, 2700)
1, 309.11s
2, 136.03s
4, 46.80s
8, 14.71s
16, 8.45s
32, 5.52s
Img data shape = (4800, 2700)
Dictionary shape = (300, 2700)
1, 75.13s
2, 33.87s
4, 12.84s
8, 3.73s
16, 2.00s
32, 1.26s
Sparse encoding done in 651.99s
Reconstructing...
Recon error...
Smallest recon error: 6.090(alpha = 1.0)
Smallest recon error: 5.137(alpha = 1.0)
/Users/don/Documents/patch_data/V46a1e-2_lars.p
Processing V46a01
Sparse encoding...
Img da